In [ ]:
__author__ = "Chuhang Liu"

import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
import scipy.signal
import scipy
from matplotlib import figure
import h5py
import hdf5plugin


In [17]:
# Define source dict
dirstr = 'D:\\2100f_data_DECTRIS\\YIG_MIT_20240320\\50um_s3\\x500k\\'
# Define working dict
dest_dir = 'D:\\2100f_data_DECTRIS\\YIG_MIT_20240320\\50um_s3\\x500k\\im\\'

In [19]:
from tqdm import trange
# Define number of image pixels
Npixel = 512
# Define number of slides
Nslide = 120
# define crosshair ratio
r_crosshair = 0.8886
# Define a 3D array for raw data
data3d = np.zeros((Npixel,Npixel,Nslide))

name = 'v1_HRIM_500k'
# Load data
for i in trange(Nslide):
    num = i + 1
    #print(i)
    if num >= 1000:
        filename = name + '_data_00' + str(num)
    elif num >= 100:
        filename = name + '_data_000' + str(num)
    elif num >= 10:
        filename = name + '_data_0000' + str(num)
    else:
        filename = name + '_data_00000' + str(num)
    with h5py.File(dirstr + filename + '.h5', 'r') as f:
        # Get the dataset you want to read
        dataset = f['entry/data/data']
        # Read the dataset into a NumPy array
        data = np.array(dataset)
        # For images with dead pixels
        #data[data == 4294967295] = 0
        data = data.reshape(512, 512)
        # fix the crosshair mark
        #print(data[254:258, 254:258])
        data[254:258, :] = data[254:258, :]/r_crosshair
        data[:, 254:258] = data[:, 254:258]/r_crosshair
        # detect dead pixels position
        '''
        for i in range(512):
            for j in range(512):
                
                if data[i, j] == 0:
                    print(i, j)
                    data[i, j] = (data[i-1, j] + data[i+1, j] + data[i, j-1] + data[i, j+1])/4

        '''
        data[164, 87] = (data[163, 87] + data[165, 87] + data[164, 86] + data[164, 88])/4
        data[192,85] = (data[191, 85] + data[193, 85] + data[192, 84] + data[192, 86])/4
        # max_threshold = 0.6 * max_val
        # min_threshold = 0.2 * max_val
        max_val = np.max(data)
        #print("max_val: ", max_val)
        min_val = np.min(data) 
        #print("min_val: ", min_val)
        # for image
        max_threshold = np.percentile(data, 99.9)
        # for diffraction
        #max_threshold = np.percentile(data, 99.9)
        #print(max_threshold)
        data[data > max_threshold] = max_threshold
        min_threshold = np.percentile(data, 0.01)
        #print(min_threshold)
        data[data < min_threshold] = min_threshold


        data3d[:,:,i] = data
        # generate tiffs
        plt.imsave(dest_dir + str(i) + '.tif', data, format = 'tiff', cmap = plt.cm.gray)

# Save raw data
#np.save(dest_dir + 'data3d_raw.npy', data3d)

  0%|          | 0/120 [00:00<?, ?it/s]

100%|██████████| 120/120 [00:02<00:00, 49.17it/s]
